# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Для начала загрузим и установим необходимые библиотеки.

In [ ]:
!pip3 install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
import time
import pandas as pd 
import numpy as np 
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

## Подготовка данных

Первичный анализ данных

In [ ]:
df = pd.read_csv('/datasets/autos.csv')
print(df.shape)
df.head()

(354369, 16)


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [ ]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [ ]:
df.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Исходя из описания признаков, можно выделить то, что некоторые из них  являются неинформативными. Удалим их.

In [ ]:
df = df.drop(['DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen'], axis = 1)
df.shape

(354369, 12)

Разделим данные на две выборки. На df будем использоваться для обучения и тестирования моделей, подборки гиперпараметров. А holdout - отложенная выборка, для расчетов метрики качества модели.

In [ ]:
df, holdout = train_test_split(df, test_size = 0.2, random_state = 333)

y = df['Price']
X = df.drop('Price', axis = 1)
y_holdout = holdout['Price']
X_holdout = holdout.drop('Price', axis = 1)

Теперь y и X разделим на train и test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3,
                                                    random_state = 333)

Мы разделили выборку перед заполнением пропусков, потому что скейлеры и энкодеры нужно обучать на тестовой части, чтобы не допустить утечку данных.

Теперь разберемся с пропусками.

In [ ]:
df.isnull().sum()

Price                    0
VehicleType          30095
RegistrationYear         0
Gearbox              15899
Power                    0
Model                15862
Kilometer                0
RegistrationMonth        0
FuelType             26423
Brand                    0
NotRepaired          57066
NumberOfPictures         0
dtype: int64

Признаки есть только в категориальных фичах. Их относительно немного, поэтому будет нерационально удалять целые признаки. Заполним пропуски наиболее часто встречающимися значениями.

In [ ]:
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']

imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_columns] = imputer.fit_transform(X_train[cat_columns])

In [ ]:
X_train.isnull().sum()

VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
NumberOfPictures     0
dtype: int64

In [ ]:
X_holdout[cat_columns] = imputer.transform(X_holdout[cat_columns])
X_test[cat_columns] = imputer.transform(X_test[cat_columns])

In [ ]:
X_test.isnull().sum()

VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
NumberOfPictures     0
dtype: int64

In [ ]:
holdout.isnull().sum()

Price                    0
VehicleType           7395
RegistrationYear         0
Gearbox               3934
Power                    0
Model                 3843
Kilometer                0
RegistrationMonth        0
FuelType              6472
Brand                    0
NotRepaired          14088
NumberOfPictures         0
dtype: int64

Числовые признаки в данном случае не имеет смысла масштабировать, потому что они предсавляют из себя либо год, либо номер месяца и тд.

Теперь закодируем категориальные фичи для корректной работы моделей.

In [ ]:
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired', 'Brand']

encoder = LabelEncoder()
for c in cat_columns:
    X_train[c] = encoder.fit_transform(X_train[c])
    X_test[c] = encoder.transform(X_test[c])
    X_holdout[c] = encoder.transform(X_holdout[c])

In [ ]:
X_train.head()

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures
307455,5,2006,1,75,116,125000,2,6,38,0,0
113490,7,2001,1,96,42,150000,7,2,24,0,0
57035,4,2018,1,0,173,150000,2,6,38,0,0
263285,7,2003,0,204,95,150000,8,2,20,0,0
62527,7,2004,1,122,59,150000,1,2,20,0,0


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198446 entries, 307455 to 149388
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   VehicleType        198446 non-null  int64
 1   RegistrationYear   198446 non-null  int64
 2   Gearbox            198446 non-null  int64
 3   Power              198446 non-null  int64
 4   Model              198446 non-null  int64
 5   Kilometer          198446 non-null  int64
 6   RegistrationMonth  198446 non-null  int64
 7   FuelType           198446 non-null  int64
 8   Brand              198446 non-null  int64
 9   NotRepaired        198446 non-null  int64
 10  NumberOfPictures   198446 non-null  int64
dtypes: int64(11)
memory usage: 18.2 MB


## Обучение моделей

**LightGBM**

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.5,
    "num_leaves": 128,  
    "max_bin": 512,
}
lgb_train = lgb.Dataset(X_train, y_train)

In [ ]:
start_time = time.time()
lgb_model = lgb.train(hyper_params, lgb_train, num_boost_round=10, verbose_eval=False)
lgb_train_time = time.time() - start_time

In [ ]:
start_time = time.time()
y_predicted = lgb_model.predict(X_test)
lgb_test_time = time.time() - start_time

In [ ]:
lgb_rmse = math.sqrt(mean_squared_error(y_test, y_predicted))
print(lgb_rmse)

1894.4993441143047


**CatBoost**

In [ ]:
catboost_model = CatBoostRegressor(iterations=50)

In [ ]:
start_time = time.time()
catboost_model.fit(X_train, y_train, verbose_eval=False)
cat_train_time = time.time() - start_time

In [ ]:
start_time = time.time()
y_predicted = catboost_model.predict(X_test)
cat_test_time = time.time() - start_time

In [ ]:
cat_rmse = math.sqrt(mean_squared_error(y_test, y_predicted))
print(cat_rmse)

1921.9413764790481


**Линейная регрессия**

In [ ]:
reg = LinearRegression()

In [ ]:
start_time = time.time()
reg.fit( X_train, y_train)
reg_train_time = time.time() - start_time

In [ ]:
start_time = time.time()
y_predicted = reg.predict(X_test)
reg_test_time = time.time() - start_time

In [ ]:
reg_rmse = math.sqrt(mean_squared_error(y_test, y_predicted))
print(reg_rmse)

3703.1209608105905


## Анализ моделей

Сравним точности моделей на X_test

In [ ]:
models = pd.DataFrame({
    'Model': ['LightGBM','CatBoost','LinReg'],
    'RMSE': [lgb_rmse, cat_rmse, reg_rmse]})

models.sort_values(by='RMSE')

,Model,RMSE
0,LightGBM,1894.499344
1,CatBoost,1921.941376
2,LinReg,3703.120961


В этом плане лучше всего показала себя модель LightGBM

Теперь сравним по скорости обучения

In [ ]:
models = pd.DataFrame({
    'Model': ['LightGBM','CatBoost','LinReg'],
    'Train_time': [lgb_train_time, cat_train_time, reg_train_time]})

models.sort_values(by='Train_time')

,Model,Train_time
2,LinReg,0.104125
0,LightGBM,0.651799
1,CatBoost,1.526968


Как и предполагалось, обычная модель линейной регрессии справилась быстрее всех.

Теперь сравним по скорости предсказания

In [ ]:
models = pd.DataFrame({
    'Model': ['LightGBM','CatBoost','LinReg'],
    'Pred_time': [lgb_test_time, cat_test_time, reg_test_time]})

models.sort_values(by='Pred_time')

,Model,Pred_time
2,LinReg,0.019107
1,CatBoost,0.027017
0,LightGBM,0.072428


Резкльтат тот же.

На основе полученных данным можно сделать вывод, что LightGBM справилась относительно лучше других. Проверим ее на holdout.

In [ ]:
pred = lgb_model.predict(X_holdout)
rmse = math.sqrt(mean_squared_error(y_holdout, pred))
print(rmse)

1889.137132811777
